<a href="https://colab.research.google.com/github/sentongo-web/MASTERS_DEGREE_PROJECT_MSDS/blob/main/data_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Mounting the drive since dataset is saved on the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#setting path for the data dictionary and the dataset
file_path = '/content/drive/My Drive/uganda_imports.xlsx'

In [5]:
# importing libraries to be used in the project
import pandas as pd
import numpy as np
import copy
import time
import pickle
from collections import defaultdict
from itertools import islice, combinations
from datetime import datetime as dt
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
import matplotlib.pyplot as plt
from ctgan import CTGAN

In [6]:
#Reading the dataset and the data dictionary
data = pd.read_excel(file_path)

In [7]:
data.head(5)

,HS_Code,Country_of_Origin,Gross_Mass,Net_Mass,Item_Price,CIF_Value,Duties_Taxes,Unit_Price_Local,Valuation_Method,Invoice_Amount_NMU,Invoice_Currency_Code,Added_Costs,Internal_Freight_NMU,Additional_Charges_NMU
0,85441900.0,CN,927.72,920.43,806.92,806.92,1035.37,NaN,NaN,806.92,USD,0.0,0.0,0.0
1,69010000.0,CN,21399.86,20000.00,2366.95,2366.95,380.94,NaN,NaN,2366.95,USD,0.0,0.0,0.0
2,83023000.0,CN,77.82,50.00,308.73,308.73,74.44,NaN,NaN,308.73,USD,0.0,0.0,0.0
3,94015000.0,CN,194.54,150.00,1285.39,1285.39,447.56,257.078,NaN,1285.39,USD,0.0,0.0,0.0
4,94031000.0,CN,194.54,150.00,1344.86,1344.86,468.26,268.972,NaN,1344.86,USD,0.0,0.0,0.0


In [8]:
# sample exchange rates to UGX (assuming these are provided)
exchange_rates = {
    'USD': 3700,   # sample rate
    'EUR': 4000,   # sample rate
    # Would add other necessary exchange rates here
}

In [9]:
# Convert all monetary columns to UGX
monetary_columns = ['Item_Price', 'CIF_Value', 'Duties_Taxes', 'Unit_Price_Local',
                    'Invoice_Amount_NMU', 'Added_Costs', 'Internal_Freight_NMU', 'Additional_Charges_NMU']

In [10]:
for column in monetary_columns:
    # since VIT_INV_CUR_COD indicates the currency of the monetary columns
    data[column] = data.apply(lambda row: row[column] * exchange_rates.get(row['Invoice_Currency_Code'], 1), axis=1)

In [11]:
# Drop the currency code column as all values are now in UGX
data.drop(columns=['Invoice_Currency_Code'], inplace=True)

In [12]:
data.head()

,HS_Code,Country_of_Origin,Gross_Mass,Net_Mass,Item_Price,CIF_Value,Duties_Taxes,Unit_Price_Local,Valuation_Method,Invoice_Amount_NMU,Added_Costs,Internal_Freight_NMU,Additional_Charges_NMU
0,85441900.0,CN,927.72,920.43,2985604.0,2985604.0,3830869.0,NaN,NaN,2985604.0,0.0,0.0,0.0
1,69010000.0,CN,21399.86,20000.00,8757715.0,8757715.0,1409478.0,NaN,NaN,8757715.0,0.0,0.0,0.0
2,83023000.0,CN,77.82,50.00,1142301.0,1142301.0,275428.0,NaN,NaN,1142301.0,0.0,0.0,0.0
3,94015000.0,CN,194.54,150.00,4755943.0,4755943.0,1655972.0,951188.6,NaN,4755943.0,0.0,0.0,0.0
4,94031000.0,CN,194.54,150.00,4975982.0,4975982.0,1732562.0,995196.4,NaN,4975982.0,0.0,0.0,0.0


In [13]:
data.shape

(1048575, 13)

In [14]:
data.columns

Index(['HS_Code', 'Country_of_Origin', 'Gross_Mass', 'Net_Mass', 'Item_Price',
       'CIF_Value', 'Duties_Taxes', 'Unit_Price_Local', 'Valuation_Method',
       'Invoice_Amount_NMU', 'Added_Costs', 'Internal_Freight_NMU',
       'Additional_Charges_NMU'],
      dtype='object')

In [17]:
# Select columns to use (renamed columns)
columns_to_keep = [
    'HS_Code',           # Harmonized system commodity code (HS code)
    'Country_of_Origin',    # Country of origin (code)
    'Gross_Mass',        # Gross mass (including packaging)
    'Net_Mass',          # Net mass (excluding packaging)
    'Item_Price',        # Item price
    'CIF_Value',         # CIF (Cost, Insurance, and Freight) value in local currency
    'Duties_Taxes',        # Duties and taxes amount for the item
    'Unit_Price_Local',        # Unit price in local currency
    'Valuation_Method',        # Valuation method
    'Invoice_Amount_NMU',    # Invoice amount in national monetary units
]

In [18]:
df_org = data[columns_to_keep]

# Since CTGAN cannot handle large input size, we sample 3000 Train Data from Source Data
df_sample = df_org.sample(3000, replace=False)
df_sample.to_csv('df_sample.xlsx', index=False, encoding='utf-8-sig')